In [74]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse
import joblib 

In [75]:
DfTrain = pd.read_csv('Otro_train.csv', encoding = 'utf-8')
DfTest = pd.read_csv('Otro_test.csv', encoding = 'utf-8')
Dfevalua = pd.read_csv('Otro_evalua.csv', encoding = 'utf-8')

In [76]:
DfTest.head()

,FECHA_ACCIDENTE,Total
0,2018-01-01,11
1,2018-01-02,15
2,2018-01-03,7
3,2018-01-04,12
4,2018-01-05,21


In [77]:
df_festivos = pd.read_csv('festivosm.csv', encoding = 'utf-8', sep =';')

In [78]:
df_festivos['Fecha'] = pd.to_datetime(df_festivos['Fecha'], format="%m/%d/%Y")

In [79]:
DfTrain['FECHA_ACCIDENTE'] = pd.to_datetime(DfTrain['FECHA_ACCIDENTE'], format="%Y/%m/%d")
DfTest['FECHA_ACCIDENTE'] = pd.to_datetime(DfTest['FECHA_ACCIDENTE'], format="%Y/%m/%d")
Dfevalua['FECHA_ACCIDENTE'] = pd.to_datetime(Dfevalua['FECHA_ACCIDENTE'], format="%Y/%m/%d")

In [80]:
DfTrain['festivo'] = DfTrain['FECHA_ACCIDENTE'].apply(lambda x: 1 if x in df_festivos['Fecha'].unique() else 0)
DfTest['festivo'] = DfTest['FECHA_ACCIDENTE'].apply(lambda x: 1 if x in df_festivos['Fecha'].unique() else 0)
Dfevalua['festivo'] = Dfevalua['FECHA_ACCIDENTE'].apply(lambda x: 1 if x in df_festivos['Fecha'].unique() else 0)

In [81]:
DfTrain['Year'] = DfTrain['FECHA_ACCIDENTE'].dt.year
DfTrain['Month'] = DfTrain['FECHA_ACCIDENTE'].dt.month
DfTrain['DayMo'] = DfTrain['FECHA_ACCIDENTE'].dt.day
DfTrain['Dayw'] = DfTrain['FECHA_ACCIDENTE'].apply(lambda x: x.strftime('%A'))

In [82]:
DfTest['Year'] = DfTest['FECHA_ACCIDENTE'].dt.year
DfTest['Month'] = DfTest['FECHA_ACCIDENTE'].dt.month
DfTest['DayMo'] = DfTest['FECHA_ACCIDENTE'].dt.day
DfTest['Dayw'] = DfTest['FECHA_ACCIDENTE'].apply(lambda x: x.strftime('%A'))

In [83]:
Dfevalua['Year'] = Dfevalua['FECHA_ACCIDENTE'].dt.year
Dfevalua['Month'] = Dfevalua['FECHA_ACCIDENTE'].dt.month
Dfevalua['DayMo'] = Dfevalua['FECHA_ACCIDENTE'].dt.day
Dfevalua['Dayw'] = Dfevalua['FECHA_ACCIDENTE'].apply(lambda x: x.strftime('%A'))

In [84]:
DfTest.head()

,FECHA_ACCIDENTE,Total,festivo,Year,Month,DayMo,Dayw
0,2018-01-01,11,1,2018,1,1,Monday
1,2018-01-02,15,0,2018,1,2,Tuesday
2,2018-01-03,7,0,2018,1,3,Wednesday
3,2018-01-04,12,0,2018,1,4,Thursday
4,2018-01-05,21,0,2018,1,5,Friday


In [85]:
def quincena(f):
    z = []
    for i,j in zip(f['DayMo'],f['Dayw']):
        if (i in [15,30,31] and j in ['Monday','Tuesday', 'Thursday', 'Friday','Wednesday']):
            z.append(1)
        else:
            z.append(0)
    return z

In [86]:
DfTrain['Quincena'] = quincena(DfTrain)
DfTest['Quincena'] = quincena(DfTest)
Dfevalua['Quincena'] = quincena(Dfevalua)

In [87]:
variab = ['DayMo','Month','Year','festivo','Dayw','Total','Quincena']

In [88]:
DfTrain = DfTrain[variab]
DfTest = DfTest[variab]
Dfevalua = Dfevalua[variab]

In [89]:
Dfevalua.head()

,DayMo,Month,Year,festivo,Dayw,Total,Quincena
0,1,1,2020,1,Wednesday,13,0
1,2,1,2020,0,Thursday,8,0
2,3,1,2020,0,Friday,7,0
3,4,1,2020,0,Saturday,11,0
4,5,1,2020,0,Sunday,5,0


## One hot encoding y Modelo

In [90]:
DfTrain = pd.get_dummies(DfTrain)
DfTest = pd.get_dummies(DfTest)
Dfevalua = pd.get_dummies(Dfevalua)

In [91]:
num = list(DfTrain.columns.values)
num.remove('Total')

In [92]:
exp_reg101 = setup(data = DfTrain, target = 'Total',
                   numeric_features = num , normalize = True, session_id = 8301)

,Description,Value
0,session_id,8301
1,Target,Total
2,Original Data,"(1276, 13)"
3,Missing Values,False
4,Numeric Features,12
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(893, 12)"


In [93]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,3.7797,23.8217,4.8490,0.0748,0.3784,0.3991,0.0220
ridge,Ridge Regression,3.7916,23.8513,4.8529,0.0725,0.3784,0.3985,0.0150
lar,Least Angle Regression,3.7917,23.8522,4.8530,0.0725,0.3784,0.3985,0.0250
lr,Linear Regression,3.8080,23.9458,4.8622,0.0696,0.3791,0.3998,1.5650
huber,Huber Regressor,3.7729,24.0604,4.8759,0.0638,0.3753,0.3857,0.0150
gbr,Gradient Boosting Regressor,3.7889,23.9521,4.8745,0.0572,0.3776,0.3923,0.0600
omp,Orthogonal Matching Pursuit,3.8192,24.4734,4.9157,0.0498,0.3835,0.4060,0.0200
en,Elastic Net,3.8292,24.9074,4.9571,0.0349,0.3882,0.4115,0.0230
lasso,Lasso Regression,3.8671,25.4868,5.0145,0.0125,0.3929,0.4170,0.0220
rf,Random Forest Regressor,3.8941,25.2166,5.0010,0.0045,0.3848,0.4005,0.2620


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [94]:
rfj = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,4.0679,27.4282,5.2372,-0.0196,0.3687,0.3537
1,3.2790,19.6834,4.4366,-0.0747,0.3869,0.4305
2,3.6157,20.8763,4.5691,-0.1407,0.3556,0.3436
3,4.8354,35.3825,5.9483,-0.2205,0.4854,0.5684
4,4.0166,24.7359,4.9735,0.1213,0.3720,0.3840
5,3.5085,22.2774,4.7199,0.0597,0.3464,0.3250
6,4.1885,31.4427,5.6074,0.2178,0.4297,0.4710
7,3.7233,21.8903,4.6787,-0.0616,0.3377,0.3297
8,3.8475,25.6163,5.0613,0.0420,0.4083,0.4264
9,3.8589,22.8333,4.7784,0.1210,0.3575,0.3726


In [95]:
tuned_rfj = tune_model(rfj)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,3.9033,24.0829,4.9074,0.1047,0.3618,0.3677
1,3.1001,18.5934,4.3120,-0.0152,0.3800,0.4277
2,3.2042,16.6022,4.0746,0.0929,0.3173,0.3060
3,4.3752,29.7054,5.4503,-0.0246,0.4621,0.5354
4,3.8863,23.3824,4.8355,0.1694,0.3725,0.3922
5,3.4990,20.9458,4.5767,0.1159,0.3310,0.3221
6,4.1732,33.0713,5.7508,0.1773,0.4363,0.4716
7,3.4074,18.6837,4.3225,0.0939,0.3102,0.3024
8,3.7510,23.2493,4.8218,0.1305,0.4036,0.4416
9,3.9491,23.0788,4.8040,0.1115,0.3582,0.3740


In [96]:
tuned_rfj

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0005,
                      min_impurity_split=None, min_samples_leaf=6,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=-1, oob_score=False,
                      random_state=8301, verbose=0, warm_start=False)

In [97]:
predict_model(tuned_rfj)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,3.8173,24.0989,4.9091,0.0720,0.3691,0.3705


,DayMo,Month,Year,festivo,Quincena,Dayw_Friday,Dayw_Monday,Dayw_Saturday,Dayw_Sunday,Dayw_Thursday,Dayw_Tuesday,Dayw_Wednesday,Total,Label
0,0.943372,-1.166563,1.249634,-0.216549,-0.233044,2.538044,-0.407181,-0.42753,-0.433013,-0.395899,-0.395899,-0.403436,12.0,13.954689
1,1.633230,0.007560,-0.679360,-0.216549,4.291042,-0.394004,-0.407181,-0.42753,-0.433013,2.525899,-0.395899,-0.403436,16.0,14.136487
2,0.598443,-1.166563,1.249634,-0.216549,-0.233044,-0.394004,-0.407181,-0.42753,-0.433013,-0.395899,2.525899,-0.403436,12.0,13.809717
3,0.943372,1.475214,-0.679360,-0.216549,-0.233044,-0.394004,-0.407181,-0.42753,-0.433013,2.525899,-0.395899,-0.403436,15.0,13.099737
4,1.058348,-1.460093,-0.679360,-0.216549,-0.233044,-0.394004,-0.407181,-0.42753,-0.433013,-0.395899,-0.395899,2.478709,14.0,12.730514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,1.403277,-0.285971,0.285137,-0.216549,-0.233044,-0.394004,-0.407181,-0.42753,-0.433013,-0.395899,2.525899,-0.403436,13.0,13.814008
379,-0.896248,1.475214,-0.679360,4.617900,-0.233044,-0.394004,-0.407181,-0.42753,-0.433013,-0.395899,2.525899,-0.403436,7.0,11.395385
380,-1.011225,0.594621,0.285137,-0.216549,-0.233044,-0.394004,-0.407181,-0.42753,-0.433013,-0.395899,-0.395899,2.478709,17.0,14.094472
381,-0.666296,0.007560,1.249634,-0.216549,-0.233044,-0.394004,2.455910,-0.42753,-0.433013,-0.395899,-0.395899,-0.403436,18.0,13.808964


In [98]:
labeltr = DfTrain['Total']
labeltest = DfTest['Total']
DfTrain = DfTrain.drop('Total', axis = 1)
DfTest = DfTest.drop('Total', axis = 1)

In [99]:
forest = tuned_rfj.fit(DfTrain,labeltr)

## Rmse entrenamiento

In [100]:
ypredTrain = forest.predict(DfTrain)

In [101]:
(mse(ypredTrain,labeltr))**(1/2)

4.588636275916727

## Rmse Test

In [102]:
ypredTest = forest.predict(DfTest)

In [103]:
(mse(ypredTest,labeltest))**(1/2)

5.22517709815033

## Rmse Evaluación

In [104]:
labelevalua = Dfevalua['Total']

In [105]:
Dfevalua = Dfevalua.drop(['Total'], axis = 1)

In [106]:
yevalua = forest.predict(Dfevalua)

In [107]:
(mse(yevalua,labelevalua))**(1/2)

6.7031670785443955

## Se guarda modelo

In [108]:
joblib.dump(forest, 'modelo_Otro_entrenado.pkl') # Se guarda el modelo.

['modelo_Otro_entrenado.pkl']